In [2]:
!pip install wandb

     |████████████████████████████████| 1.7 MB 4.2 MB/s 
     |████████████████████████████████| 97 kB 4.9 MB/s 
     |████████████████████████████████| 180 kB 43.6 MB/s 
     |████████████████████████████████| 140 kB 47.2 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=347d23164f5681d36b63e55ba3bbb4646c2d6d84e0edd275086065e4d8673310
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=5d1b1710685d32c37cb680ab8fc54fd5941013a517546bcfcfa8eab8e67766a6
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools


In [3]:
import torch
import torchvision
import wandb

In [4]:
def calc_acc(preds,labels):
  _,preds_max = torch.max(preds, 1)
  acc = torch.sum(preds_max==labels.data,dtype = torch.float64)/len(preds)
  return acc

In [5]:
class MyModel(torch.nn.Module):
  
   
  def __init__(self): 
     super().__init__()

     self.fc1= torch.nn.Linear(784,256)
     self.fc2 = torch.nn.Linear(256,128)
     self.fc3 = torch.nn.Linear(128,10)
    

  def forward(self ,x):
    #input shape 32*28*28
    x = x.reshape((x.shape[0],784))
    #output shape: 32*784
    x = self.fc1(x)
    x = torch.relu(x)
    x = self.fc2(x)
    x = torch.relu(x)
    x = torch.dropout(x, 0.2 , train=True)
    x = self.fc3(x)
    x = torch.softmax(x, dim=1)
    return x

In [6]:
# device =torch.device("cpu")
device = torch.device("cuda")
model = MyModel()
model = model.to(device) 
model.train(True)

MyModel(
  (fc1): Linear(in_features=784, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
)

In [7]:
# Hyper parameter
batch_size=64
epoch=20
config = wandb.config
config.learning_rate = 0.001

In [8]:
wandb.init(project="fashion mnist")

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [9]:
#Data Preparing
transform = torchvision.transforms.Compose([
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize((0),(1))
            # torchvision.transforms.RandomHorizontalFILIP(),              
])

dataset = torchvision.datasets.FashionMNIST("./dataset",train =True,download=True, transform = transform )
train_data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)


  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./dataset/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./dataset/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./dataset/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./dataset/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ./dataset/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./dataset/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ./dataset/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./dataset/FashionMNIST/raw



In [10]:
#compile
optimizer = torch.optim.Adam(model.parameters(),lr=config.learning_rate)
loss_function = torch.nn.CrossEntropyLoss()

In [11]:
#train_model
for epochs in range(epoch):
    train_loss = 0.0
    train_acc =0.0
    for images , labels in train_data_loader:
      images = images.to(device)
      labels = labels.to(device)
      optimizer.zero_grad()
      # 1-forwarding
      preds = model(images) 
      # 2- backwarding
      loss = loss_function(preds,labels)
      loss.backward()
      # 3- update
      optimizer.step()
      train_acc+= calc_acc(preds, labels)
      train_loss+=loss
    total_loss = train_loss/len(train_data_loader)
    total_acc = train_acc/len(train_data_loader)
    
    print(f"Epoch:{epochs}: Loss:{total_loss}" )
    print(f"Acc:{epochs}: acc:{total_acc}" )
    print("-------------------------------------")

    wandb.log({'Epoch':  epochs + 1,
              'Loss': total_loss,
              'Acc': total_acc
                              })

Epoch:0: Loss:1.734573483467102
Acc:0: acc:0.7339085820895522
-------------------------------------
Epoch:1: Loss:1.678455114364624
Acc:1: acc:0.7834821428571428
-------------------------------------
Epoch:2: Loss:1.6686190366744995
Acc:2: acc:0.7923440831556503
-------------------------------------
Epoch:3: Loss:1.6630700826644897
Acc:3: acc:0.7980743603411514
-------------------------------------
Epoch:4: Loss:1.6574816703796387
Acc:4: acc:0.8031549840085288
-------------------------------------
Epoch:5: Loss:1.6529513597488403
Acc:5: acc:0.8081523187633262
-------------------------------------
Epoch:6: Loss:1.6528958082199097
Acc:6: acc:0.8080690298507462
-------------------------------------
Epoch:7: Loss:1.6481364965438843
Acc:7: acc:0.8130163912579957
-------------------------------------
Epoch:8: Loss:1.647605299949646
Acc:8: acc:0.8133495469083155
-------------------------------------
Epoch:9: Loss:1.6465750932693481
Acc:9: acc:0.8142490671641791
-------------------------------

In [14]:
torch.save(model.state_dict(),"fashion_mnist")

In [15]:
import cv2
import numpy as np

model.eval()

img = cv2.imread("/content/nG8Kj.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

img = cv2.resize(img,(28,28))
tensor = transform(img).unsqueeze(0).to(device)

preds = model(tensor)

preds = preds.cpu().detach().numpy()

output = np.argmax(preds)
output

8